In [ ]:
# Install required packages in Jupyter
!pip install fastapi uvicorn nest_asyncio sqlalchemy passlib[bcrypt] python-multipart pillow jose

# -------------------------------------------
# Imports & Setup
# -------------------------------------------
import nest_asyncio
nest_asyncio.apply()

from fastapi import FastAPI, UploadFile, File, Depends, HTTPException, status
from fastapi.security import OAuth2PasswordBearer, OAuth2PasswordRequestForm
from jose import JWTError, jwt
from passlib.context import CryptContext
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey
from sqlalchemy.orm import declarative_base, sessionmaker, Session
from PIL import Image, ImageFilter, ImageOps
import io
import shutil
import os
from datetime import datetime, timedelta

# -------------------------------------------
# Database Setup
# -------------------------------------------
SQLALCHEMY_DATABASE_URL = "sqlite:///./test.db"
engine = create_engine(SQLALCHEMY_DATABASE_URL, connect_args={"check_same_thread": False})
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)
Base = declarative_base()

# -------------------------------------------
# User Model
# -------------------------------------------
class User(Base):
    __tablename__ = "users"
    id = Column(Integer, primary_key=True, index=True)
    username = Column(String, unique=True, index=True)
    hashed_password = Column(String)

class ImageModel(Base):
    __tablename__ = "images"
    id = Column(Integer, primary_key=True, index=True)
    filename = Column(String)
    owner_id = Column(Integer, ForeignKey("users.id"))

Base.metadata.create_all(bind=engine)

# -------------------------------------------
# JWT & Security Setup
# -------------------------------------------
SECRET_KEY = "your_secret_key"
ALGORITHM = "HS256"
ACCESS_TOKEN_EXPIRE_MINUTES = 30

pwd_context = CryptContext(schemes=["bcrypt"], deprecated="auto")
oauth2_scheme = OAuth2PasswordBearer(tokenUrl="login")

def get_db():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()

def verify_password(plain_password, hashed_password):
    return pwd_context.verify(plain_password, hashed_password)

def get_password_hash(password):
    return pwd_context.hash(password)

def create_access_token(data: dict, expires_delta: timedelta | None = None):
    to_encode = data.copy()
    expire = datetime.utcnow() + (expires_delta or timedelta(minutes=15))
    to_encode.update({"exp": expire})
    encoded_jwt = jwt.encode(to_encode, SECRET_KEY, algorithm=ALGORITHM)
    return encoded_jwt

def get_current_user(token: str = Depends(oauth2_scheme), db: Session = Depends(get_db)):
    credentials_exception = HTTPException(
        status_code=status.HTTP_401_UNAUTHORIZED,
        detail="Could not validate credentials",
    )
    try:
        payload = jwt.decode(token, SECRET_KEY, algorithms=[ALGORITHM])
        username: str = payload.get("sub")
        if username is None:
            raise credentials_exception
    except JWTError:
        raise credentials_exception
    user = db.query(User).filter(User.username == username).first()
    if user is None:
        raise credentials_exception
    return user

# -------------------------------------------
# FastAPI App
# -------------------------------------------
app = FastAPI()

# -------------------------------------------
# Register
# -------------------------------------------
@app.post("/register")
def register(form_data: OAuth2PasswordRequestForm = Depends(), db: Session = Depends(get_db)):
    user = db.query(User).filter(User.username == form_data.username).first()
    if user:
        raise HTTPException(status_code=400, detail="Username already registered")
    hashed_password = get_password_hash(form_data.password)
    new_user = User(username=form_data.username, hashed_password=hashed_password)
    db.add(new_user)
    db.commit()
    db.refresh(new_user)
    access_token = create_access_token(data={"sub": new_user.username})
    return {"access_token": access_token, "token_type": "bearer"}

# -------------------------------------------
# Login
# -------------------------------------------
@app.post("/login")
def login(form_data: OAuth2PasswordRequestForm = Depends(), db: Session = Depends(get_db)):
    user = db.query(User).filter(User.username == form_data.username).first()
    if not user or not verify_password(form_data.password, user.hashed_password):
        raise HTTPException(status_code=400, detail="Invalid credentials")
    access_token = create_access_token(data={"sub": user.username})
    return {"access_token": access_token, "token_type": "bearer"}

# -------------------------------------------
# Upload Image
# -------------------------------------------
UPLOAD_FOLDER = "./uploads"
os.makedirs(UPLOAD_FOLDER, exist_ok=True)

@app.post("/images")
def upload_image(file: UploadFile = File(...), current_user: User = Depends(get_current_user), db: Session = Depends(get_db)):
    file_location = f"{UPLOAD_FOLDER}/{file.filename}"
    with open(file_location, "wb") as buffer:
        shutil.copyfileobj(file.file, buffer)
    image = ImageModel(filename=file.filename, owner_id=current_user.id)
    db.add(image)
    db.commit()
    db.refresh(image)
    return {"id": image.id, "filename": file.filename}

# -------------------------------------------
# List Images
# -------------------------------------------
@app.get("/images")
def list_images(current_user: User = Depends(get_current_user), db: Session = Depends(get_db)):
    images = db.query(ImageModel).filter(ImageModel.owner_id == current_user.id).all()
    return [{"id": img.id, "filename": img.filename} for img in images]

# -------------------------------------------
# Retrieve Image
# -------------------------------------------
@app.get("/images/{image_id}")
def get_image(image_id: int, current_user: User = Depends(get_current_user), db: Session = Depends(get_db)):
    image = db.query(ImageModel).filter(ImageModel.id == image_id, ImageModel.owner_id == current_user.id).first()
    if not image:
        raise HTTPException(status_code=404, detail="Image not found")
    file_path = f"{UPLOAD_FOLDER}/{image.filename}"
    return {"filename": image.filename, "path": file_path}

# -------------------------------------------
# Transform Image
# -------------------------------------------
# -------------------------------------------
# Extended Transform Image
# -------------------------------------------
@app.post("/images/{image_id}/transform")
def transform_image(
    image_id: int,
    resize_width: int = 0,
    resize_height: int = 0,
    rotate: int = 0,
    grayscale: bool = False,
    crop_x: int = 0,
    crop_y: int = 0,
    crop_width: int = 0,
    crop_height: int = 0,
    flip: bool = False,
    mirror: bool = False,
    format: str = "",
    quality: int = 85,  # for compression
    watermark_text: str = "",
    current_user: User = Depends(get_current_user),
    db: Session = Depends(get_db)
):
    image = db.query(ImageModel).filter(ImageModel.id == image_id, ImageModel.owner_id == current_user.id).first()
    if not image:
        raise HTTPException(status_code=404, detail="Image not found")
    file_path = f"{UPLOAD_FOLDER}/{image.filename}"

    img = Image.open(file_path).convert("RGBA")

    # Resize
    if resize_width > 0 and resize_height > 0:
        img = img.resize((resize_width, resize_height))

    # Rotate
    if rotate != 0:
        img = img.rotate(rotate)

    # Grayscale
    if grayscale:
        img = ImageOps.grayscale(img).convert("RGBA")

    # Crop
    if crop_width > 0 and crop_height > 0:
        img = img.crop((crop_x, crop_y, crop_x + crop_width, crop_y + crop_height))

    # Flip
    if flip:
        img = ImageOps.flip(img)

    # Mirror
    if mirror:
        img = ImageOps.mirror(img)

    # Add watermark text
    if watermark_text:
        from PIL import ImageDraw, ImageFont
        draw = ImageDraw.Draw(img)
        font_size = max(20, img.width // 20)
        try:
            font = ImageFont.truetype("arial.ttf", font_size)
        except:
            font = ImageFont.load_default()
        text_position = (10, 10)
        draw.text(text_position, watermark_text, fill=(255, 255, 255, 128), font=font)

    # Convert format
    new_format = format.upper() if format else img.format or "PNG"
    if new_format not in ["JPEG", "PNG", "WEBP"]:
        new_format = "PNG"

    # New filename
    new_filename = f"transformed_{os.path.splitext(image.filename)[0]}.{new_format.lower()}"
    new_file_path = f"{UPLOAD_FOLDER}/{new_filename}"

    # Save with compression (JPEG/WebP quality)
    if new_format == "JPEG":
        img = img.convert("RGB")  # JPEG doesn't support alpha
        img.save(new_file_path, new_format, quality=quality)
    else:
        img.save(new_file_path, new_format)

    return {
        "original_image": file_path,
        "transformed_image": new_file_path,
        "format": new_format,
        "quality": quality
    }


# -------------------------------------------
# Run in Notebook
# -------------------------------------------
import uvicorn
uvicorn.run(app, host="0.0.0.0", port=8001)
